In [46]:
import requests
import pandas as pd
import mysql.connector
from mysql.connector import Error
import calendar
import os

In [37]:
test_csv = pd.read_csv('1INCHUSDT.csv')
print(test_csv.columns)
print(len(test_csv))
test_csv.head(3)

Index(['Unnamed: 0', 'timestamp', 'open', 'high', 'low', 'close', 'volume',
       'close_time', 'quote_asset_volume', 'number_of_trades',
       'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume',
       'ignore'],
      dtype='object')
33088


,Unnamed: 0,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
0,0,2020-12-25 05:00:00,0.2000,3.0885,0.2000,2.5826,13125130.77,1608875999999,3.553052e+07,48768,6316317.77,1.724763e+07,0
1,1,2020-12-25 06:00:00,2.5824,2.6900,2.2249,2.5059,9270672.93,1608879599999,2.244087e+07,31099,4094773.68,9.861481e+06,0
2,2,2020-12-25 07:00:00,2.5152,2.8870,2.3609,2.6237,7923760.56,1608883199999,2.130043e+07,33001,4311917.31,1.160785e+07,0


In [38]:
test_csv['timestamp'] = pd.to_datetime(test_csv['timestamp'])
print(max(test_csv['timestamp']))
print(min(test_csv['timestamp']))

2024-10-04 10:00:00
2020-12-25 05:00:00


In [39]:
test_csv = test_csv.head(20)

In [ ]:
import mysql.connector
from mysql.connector import Error

# MySQL database connection function
def connect_to_database():
    try:
        # Establishing connection to the database
        connection = mysql.connector.connect(
            host='crypto-matter.c5eq66ogk1mf.eu-central-1.rds.amazonaws.com',
            database='Crypto',
            user='Jing',  # Replace with your actual first name
            password='Crypto12!'
        )

        if connection.is_connected():
            db_info = connection.get_server_info()
            print("Connected to MySQL database, MySQL Server version: ", db_info)
            return connection

    except Error as e:
        print("Error while connecting to MySQL", e)
        return None

# Function to execute a SQL query and return the results as a list of tuples
def execute_select_query(connection, query, params=None):
    cursor = connection.cursor()
    try:
        if params:
            cursor.execute(query, params)
        else:
            cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"Error executing query: {e}")
        return None

# Example: Query to get all price data for a specific crypto_symbol
def get_all_price_data(connection, crypto_symbol):
    query = """
    SELECT * 
    FROM crypto_price 
    WHERE crypto_symbol = %s
    """
    return execute_select_query(connection, query, (crypto_symbol,))

# Example: Query to get highest and lowest prices for a crypto_symbol in a specific month
def get_high_low_for_month(connection, crypto_symbol, year, month):
    query = """
    SELECT 
        MAX(high) AS max_high, 
        MIN(low) AS min_low
    FROM crypto_price 
    WHERE crypto_symbol = %s
      AND YEAR(date) = %s 
      AND MONTH(date) = %s
    """
    return execute_select_query(connection, query, (crypto_symbol, year, month))

# Example: Query to get average closing price for a crypto_symbol in a specific date range
def get_average_close_price(connection, crypto_symbol, start_date, end_date):
    query = """
    SELECT 
        AVG(close) AS avg_close
    FROM crypto_price 
    WHERE crypto_symbol = %s
      AND date BETWEEN %s AND %s
    """
    return execute_select_query(connection, query, (crypto_symbol, start_date, end_date))

# Example: Query to get the most recent price for a crypto_symbol
def get_most_recent_price(connection, crypto_symbol):
    query = """
    SELECT * 
    FROM crypto_price 
    WHERE crypto_symbol = %s
    ORDER BY date DESC
    LIMIT 1
    """
    return execute_select_query(connection, query, (crypto_symbol,))

# Main function to test the queries
def main():
    # Connect to MySQL database
    connection = connect_to_database()

    if connection:
        # Example 1: Get all price data for 'BTCUSDT'
        all_data = get_all_price_data(connection, 'BTCUSDT')
        print("All price data for BTCUSDT:")
        for row in all_data:
            print(row)

        # Example 2: Get highest and lowest prices for 'BTCUSDT' in August 2023
        high_low = get_high_low_for_month(connection, 'BTCUSDT', 2023, 8)
        print("\nHighest and lowest prices for BTCUSDT in August 2023:")
        print(high_low)

        # Example 3: Get


In [7]:
# MySQL database connection function
def connect_to_database():
    try:
        # Establishing connection to the database
        connection = mysql.connector.connect(
            host='crypto-matter.c5eq66ogk1mf.eu-central-1.rds.amazonaws.com',
            database='Crypto',
            user='Jing',  # Replace with your actual first name
            password='Crypto12!'
        )

        if connection.is_connected():
            db_info = connection.get_server_info()
            print("Connected to MySQL database, MySQL Server version: ", db_info)
            return connection

    except Error as e:
        print("Error while connecting to MySQL", e)
        return None

In [11]:
connection = connect_to_database()

Connected to MySQL database, MySQL Server version:  8.0.39


In [12]:
# Function to get all table names in the database
def get_table_names(connection):
    cursor = connection.cursor()
    cursor.execute("SHOW TABLES;")
    tables = cursor.fetchall()
    return [table[0] for table in tables]

get_table_names(connection)

[bytearray(b'crypto_lending_borrowing'),
 bytearray(b'crypto_price'),
 bytearray(b'crypto_reference')]

In [48]:
# Function to get column names and their data types for a specific table
def get_column_names_and_data_types(connection, table_name):
    cursor = connection.cursor()
    
    # Run SHOW COLUMNS to get both the column names and their data types
    cursor.execute(f"SHOW COLUMNS FROM {table_name};")
    
    # Fetch the columns and their metadata
    columns = cursor.fetchall()
    
    # Create a list of tuples with column name and data type
    column_info = [(column[0], column[1]) for column in columns]
    
    return column_info

# Example usage: get column names and data types for 'crypto_price' table
column_info = get_column_names_and_data_types(connection, 'crypto_price')

# Print the column names and data types
for col_name, col_type in column_info:
    print(f"Column: {col_name}, Data Type: {col_type}")


Column: crypto_symbol, Data Type: b'varchar(10)'
Column: date, Data Type: b'datetime'
Column: high, Data Type: b'decimal(15,2)'
Column: low, Data Type: b'decimal(15,2)'
Column: close, Data Type: b'decimal(15,2)'
Column: adj_close, Data Type: b'decimal(15,2)'
Column: volume, Data Type: b'bigint'
Column: market_cap, Data Type: b'bigint'
Column: coin_supply, Data Type: b'bigint'


In [ ]:
# Function to view data from a specific table
def view_table_data(connection, table_name, limit=10):
    cursor = connection.cursor()
    query = f"SELECT * FROM {table_name} LIMIT %s;"
    cursor.execute(query, (limit,))
    data = cursor.fetchall()
    return data



In [40]:
# Function to delete a row based on column value
def delete_row_by_column_value(connection, table_name, column_name, column_value):
    cursor = connection.cursor()
    try:
        # SQL DELETE query with placeholders
        delete_query = f"DELETE FROM {table_name} WHERE {column_name} = %s;"
        cursor.execute(delete_query, (column_value,))
        connection.commit()  # Commit the transaction
        print(f"Row deleted from {table_name} where {column_name} = {column_value}")
    except Error as e:
        print(f"Error deleting data: {e}")

# Deleting the dummies
delete_row_by_column_value(connection, 'crypto_price', 'crypto_symbol', '1INCHUSDT')

Row deleted from crypto_price where crypto_symbol = 1INCHUSDT


In [34]:
test_csv.iloc[0]

Unnamed: 0                                        0
timestamp                       2020-12-25 05:00:00
open                                            0.2
high                                         3.0885
low                                             0.2
close                                        2.5826
volume                                  13125130.77
close_time                            1608875999999
quote_asset_volume                  35530516.326796
number_of_trades                              48768
taker_buy_base_asset_volume              6316317.77
taker_buy_quote_asset_volume         17247626.52057
ignore                                            0
Name: 0, dtype: object

In [35]:
# Function to store data into MySQL database (crypto_price table)
def store_to_mysql(df, connection, table_name, crypto_symbol):
    cursor = connection.cursor()

    # Step 1: Check if the crypto_symbol exists in crypto_reference table
    check_query = "SELECT crypto_symbol FROM crypto_reference WHERE crypto_symbol = %s"
    cursor.execute(check_query, (crypto_symbol,))
    result = cursor.fetchone()

    # Step 2: If the crypto_symbol does not exist, insert it into the crypto_reference table
    if result is None:
        print(f"{crypto_symbol} not found in crypto_reference, inserting now...")
        insert_reference_query = "INSERT INTO crypto_reference (crypto_symbol, crypto_name) VALUES (%s, %s)"
        cursor.execute(insert_reference_query, (crypto_symbol, crypto_symbol))  # Assuming crypto_name is the same as the symbol

    # Step 3: Insert data into crypto_price table, use ON DUPLICATE KEY UPDATE to prevent duplicates
    insert_query = f"""
    INSERT INTO {table_name} 
    (crypto_symbol, date, high, low, close, adj_close, volume, market_cap, coin_supply) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE 
    high=VALUES(high), low=VALUES(low), close=VALUES(close), adj_close=VALUES(adj_close), volume=VALUES(volume), market_cap=VALUES(market_cap), coin_supply=VALUES(coin_supply)
    """

    for row in df.itertuples(index=False):
        # Convert timestamp to string in the format MySQL expects ('YYYY-MM-DD HH:MM:SS')
        timestamp_str = row.timestamp.strftime('%Y-%m-%d %H:%M:%S')

        # Ensure the values passed match the types expected in the SQL table
        cursor.execute(insert_query, (
            crypto_symbol,               # crypto_symbol
            timestamp_str,               # date
            round(row.high, 2),          # high (decimal)
            round(row.low, 2),           # low (decimal)
            round(row.close, 2),         # close (decimal)
            round(row.close, 2),         # adj_close (use close if no adj_close column)
            int(row.volume),             # volume (bigint)
            None,                        # market_cap (assumed None as it's not in the CSV)
            None                         # coin_supply (assumed None as it's not in the CSV)
        ))

    connection.commit()
    print(f"{len(df)} records successfully inserted into {table_name} table")


In [42]:
store_to_mysql(test_csv, connection, 'crypto_price', '1INCHUSDT')

20 records successfully inserted into crypto_price table


In [43]:
view_table_data(connection, 'crypto_price')

[('1INCHUSDT',
  datetime.datetime(2020, 12, 25, 5, 0),
  Decimal('3.09'),
  Decimal('0.20'),
  Decimal('2.58'),
  Decimal('2.58'),
  13125130,
  None,
  None),
 ('1INCHUSDT',
  datetime.datetime(2020, 12, 25, 6, 0),
  Decimal('2.69'),
  Decimal('2.22'),
  Decimal('2.51'),
  Decimal('2.51'),
  9270672,
  None,
  None),
 ('1INCHUSDT',
  datetime.datetime(2020, 12, 25, 7, 0),
  Decimal('2.89'),
  Decimal('2.36'),
  Decimal('2.62'),
  Decimal('2.62'),
  7923760,
  None,
  None),
 ('1INCHUSDT',
  datetime.datetime(2020, 12, 25, 8, 0),
  Decimal('2.82'),
  Decimal('2.46'),
  Decimal('2.61'),
  Decimal('2.61'),
  6625565,
  None,
  None),
 ('1INCHUSDT',
  datetime.datetime(2020, 12, 25, 9, 0),
  Decimal('2.75'),
  Decimal('2.56'),
  Decimal('2.64'),
  Decimal('2.64'),
  3739132,
  None,
  None),
 ('1INCHUSDT',
  datetime.datetime(2020, 12, 25, 10, 0),
  Decimal('2.98'),
  Decimal('2.63'),
  Decimal('2.77'),
  Decimal('2.77'),
  7946712,
  None,
  None),
 ('1INCHUSDT',
  datetime.datetime(202

In [47]:
# looping through csv files

directory_path = "prices"

for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):
          try:
               file_path = os.path.join(directory_path, filename)
               file_name_without_extension = filename.split('.')[0]
               name = file_name_without_extension
               print(name)

               # Read the CSV file into a DataFrame
               df = pd.read_csv(file_path)
               df['timestamp'] = pd.to_datetime(df['timestamp'])
               store_to_mysql(df, connection, 'crypto_price', name)

          except Exception as e:
               print(f"An error occurred while processing {filename}: {e}")
               continue


1INCHUSDT
AAVEUSDT
ARBUSDT
BALUSDT
BATUSDT
BTCUSDT
BUSDUSDT
CAKEUSDT
CRVUSDT
DAIUSDT
ENJUSDT
ENSUSDT
ETHUSDT
FDUSDUSDT
FXSUSDT
GHSTUSDT
GNOUSDT
KNCUSDT
LDOUSDT
LENDUSDT
LINKUSDT
MANAUSDT
METISUSDT
MKRUSDT
OPUSDT
RENUSDT
REPUSDT
RPLUSDT
SNXUSDT
STGUSDT
SUSDUSDT
SUSHIUSDT
TUSDUSDT
UNIUSDT
USDCUSDT
WBTCUSDT
YFIUSDT
ZRXUSDT
